In [11]:
%reload_ext autoreload


In [12]:
import pandas as pd
from src.train import train_loop
from src.model import CNNLSTMModel
from src.data_loader import InverterTimeSeriesDataset
import torch

inverter_data = pd.read_parquet('data/inverter_data/')
inverter_data['event_local_time'] = pd.to_datetime(inverter_data['event_local_time'])
failure_sessions = pd.read_csv('data/failure_sessions.csv', index_col=0, parse_dates=['start_time', 'end_time'])

In [14]:
import sys
sys.path.append('src')        # point at your src/ folder
from preprocess import load_parquet_data, load_failure_sessions, prepare_dataset

inverter_df      = load_parquet_data('data/inverter_data')
failure_sessions = load_failure_sessions('data/failure_sessions.csv')
labeled_df       = prepare_dataset(inverter_df, failure_sessions, pre_days=5)



Loaded 1 parquet files → 185922 rows
Kept 61 sessions longer than 3 days
Total pre-failure rows: 1440
Total rows: 184802


In [15]:
from preprocess import prepare_dataset
labeled_df = prepare_dataset(inverter_df, failure_sessions, pre_days=5)
labeled_df.fillna(0, inplace=True)

#print(labeled_df.columns.tolist())


Total pre-failure rows: 1440
Total rows: 184802


In [17]:
feature_cols = ['metric.AC_CURRENT_A.MEASURED', 'metric.AC_POWER.MEASURED']

dataset = InverterTimeSeriesDataset(
    labeled_df,
    feature_cols,
    label_col='label',
    window_size=30,
    stride=1
)



Processing devices: 100%|██████████| 16/16 [00:21<00:00,  1.31s/it]


In [18]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

model = CNNLSTMModel(num_features=len(feature_cols))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

In [19]:
train_loop(model, train_loader, num_epochs=10, optimizer=optimizer, device='cpu', criterion=criterion)

[Epoch 1/10] Step 0/5760 - Loss: 0.2873
[Epoch 1/10] Step 100/5760 - Loss: 0.0005
[Epoch 1/10] Step 200/5760 - Loss: 0.0002
[Epoch 1/10] Step 300/5760 - Loss: 0.0312
[Epoch 1/10] Step 400/5760 - Loss: 0.0000
[Epoch 1/10] Step 500/5760 - Loss: 0.0000
[Epoch 1/10] Step 600/5760 - Loss: 0.0000
[Epoch 1/10] Step 700/5760 - Loss: 0.0000
[Epoch 1/10] Step 800/5760 - Loss: 0.0312
[Epoch 1/10] Step 900/5760 - Loss: 0.0000
[Epoch 1/10] Step 1000/5760 - Loss: 0.0000
[Epoch 1/10] Step 1100/5760 - Loss: 0.0001
[Epoch 1/10] Step 1200/5760 - Loss: 0.0312


KeyboardInterrupt: 